In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 512kB 32.5MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 3.8MB 17.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=2324be786a8eb9227020d8294c401088fd2af410b18d23ffac009a1256609046
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
import numpy as np

In [4]:
def get_char_embedd(char_encode):
  LEN_OF_VOCAB = 137
  shape = char_encode.shape
  char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        char_int = char_encode[i,j,k]
        char_int = char_int.astype(np.int64)
        onehot = np.zeros(LEN_OF_VOCAB)
        onehot[char_int] = 1
        char_embedd[i,j,k,:] = onehot
  
  return char_embedd

In [5]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [6]:
num_word = 42
num_char = 32
num_char_of_a_sen = num_word * num_char
char_embedding_dim = 137
batch_size = 64
output_lenght = 13
#===========================================================================================
char_input  = Input(shape=(num_word, num_char, char_embedding_dim), name="char_input")
char        = TimeDistributed(  Conv1D(filters = 8, kernel_size = 3, padding='same') ) (char_input)
char        = TimeDistributed(  Conv1D(filters = 8, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  MaxPool1D(pool_size = 2)  )  (char)
char        = TimeDistributed(  Conv1D(filters = 16, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  Conv1D(filters = 16, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  MaxPool1D(pool_size = 2)  )  (char)
char_out    = TimeDistributed(  Flatten()   )   (char)
#============================================================================================
word_input  = Input(shape=(num_word, 300), name="word_input")
word_out    = word_input
#============================================================================================
concatenated = concatenate([char_out, word_out])
# main BiLSTM model==========================================================================
out     = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(concatenated)
out     = TimeDistributed(Dense(output_lenght))(out)
output  = Activation('softmax')(out)
# crf = CRF(14, name="output")
# output = crf(out)
#===========================================================================================
m = Model(inputs=[char_input, word_input], outputs=output)
plot_model(m, "/content/drive/MyDrive/model/img/model_.png", show_shapes=True)
m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# m.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy, f1_m])
m.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 42, 32, 137)  0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 42, 32, 8)    3296        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 42, 32, 8)    200         time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 42, 16, 8)    0           time

In [7]:
from keras.models import load_model

In [8]:
import keras

In [9]:
import math
import os.path
from os import path

link_folder = "/content/drive/MyDrive/fast_text/split data/data_0"
index = 0
while path.exists(link_folder):
# while index==0:
  print("DATA {}".format(index))

  link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)
  if path.exists(link_model):
    index+=1
    link_folder = "/content/drive/MyDrive/fast_text/split data/data_{}".format(index)
    continue
  else:
    global m
    link_train = "/content/drive/MyDrive/fast_text/split data/data_{}/train/".format(index)
    link_val = "/content/drive/MyDrive/fast_text/split data/data_{}/val/".format(index)

    char_encode = np.loadtxt(link_train+'char_encode.txt')
    char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
    print(char_encode.shape)
    tag = np.loadtxt(link_train+'tag_embedd.txt')
    tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
    print(tag.shape)
    word = np.loadtxt(link_train+'word_embedd.txt')
    word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
    print(word.shape)

    val_char_encode = np.loadtxt(link_val+'char_encode.txt')
    val_char_encode = val_char_encode.reshape(int(val_char_encode.shape[0]/42), 42, val_char_encode.shape[1])
    print(val_char_encode.shape)
    val_tag = np.loadtxt(link_val+'tag_embedd.txt')
    val_tag = val_tag.reshape(int(val_tag.shape[0]/42), 42, val_tag.shape[1])
    print(val_tag.shape)
    val_word = np.loadtxt(link_val+'word_embedd.txt')
    val_word = val_word.reshape(int(val_word.shape[0]/42), 42, val_word.shape[1])
    print(val_word.shape)

    char_embedd = get_char_embedd(char_encode)
    print(char_embedd.shape)
    val_char_embedd = get_char_embedd(val_char_encode)
    print(val_char_embedd.shape)
    
    link_old_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index-1)
    if path.exists(link_old_model):
      m = load_model(link_old_model)

    cp_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/CPoint model/model-data-check-point-{}.h5'.format(index), verbose=1, save_best_only=True)
    es_callback = keras.callbacks.EarlyStopping(monitor='acc', patience=3)

    history = m.fit([char_embedd, word], tag,  
          epochs=100,
          batch_size = 64,
          validation_data=([val_char_embedd, val_word], val_tag), 
          callbacks=[cp_callback,es_callback]
    )

    print(link_old_model)
    print(link_model)
    m.save(link_model)

    index+=1
    link_folder = "/content/drive/MyDrive/fast_text/split data/data_{}".format(index)

DATA 0
(3150, 42, 32)
(3150, 42, 13)
(3150, 42, 300)
(450, 42, 32)
(450, 42, 13)
(450, 42, 300)
(3150, 42, 32, 137)
(450, 42, 32, 137)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3150 samples, validate on 450 samples
Epoch 1/100





3150/3150 [==============================] - 40s 13ms/step - loss: 1.0269 - acc: 0.7299 - val_loss: 0.6556 - val_acc: 0.7922

Epoch 00001: val_loss improved from inf to 0.65557, saving model to /content/drive/MyDrive/CPoint model/model-data-check-point-0.h5
Epoch 2/100
3150/3150 [==============================] - 38s 12ms/step - loss: 0.5014 - acc: 0.8384 - val_loss: 0.3372 - val_acc: 0.8835

Epoch 00002: val_loss improved from 0.65557 to 0.33721, saving model to /content/drive/MyDrive/CPoint model/model-data-check-point-0.h5
Epoch 3/100
3150/3150 [==============================] - 38s 12ms/step - loss: 0.2383 - acc: 0.9204 - val_loss: 0.1399 - val_acc: 0.9556

Epoch 00003: val_loss improved fro